# Agent6

## agent 5 기반 json spliter 추가

In [1]:
import os
import torch
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType                    
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv

# 추가된 import
from typing import TypedDict
from langchain_upstage import UpstageGroundednessCheck
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
from langgraph.errors import GraphRecursionError
from langchain.schema import Document  # 추가된 import

# .env 파일 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 로드
openai_api_key = os.getenv("OPENAI_API_KEY")




In [2]:
# JSON 파일 경로 설정
file_path = 'merged_data.json'

# JSON 파일을 열고 로드
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# JSON 객체를 문자열로 변환
json_string = str(data)

# 특정 기준 문자열을 사용하여 분리
split_docs = json_string.split('}}')

# Langchain Document 객체로 변환
documents = [Document(page_content=doc.strip() + '}}') for doc in split_docs if doc.strip()]

# 분리된 Document 출력 (필요 시)
# for i, doc in enumerate(documents):
#     print(f"Document {i + 1}:\n{doc.page_content}\n")

# 문서 분할 및 임베딩 생성
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)


In [3]:
# CUDA 사용 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 임베딩 모델 설정 (GPU 사용)
embedding_model_name = 'jhgan/ko-sroberta-multitask'
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': device}
)

# FAISS 인덱스 설정
index_path = 'faiss_index'

# VectorStore 생성 또는 로드
if os.path.exists(index_path):
    print("저장된 FAISS 인덱스를 로드합니다...")
    vectorstore = FAISS.load_local(
        index_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
else:
    print("FAISS 인덱스를 생성합니다...")
    vectorstore = FAISS.from_documents(split_docs, embedding_model)
    vectorstore.save_local(index_path)


/tmp/ipykernel_290/1672806552.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/home/elicer/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


저장된 FAISS 인덱스를 로드합니다...


In [4]:
# Retriever 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# 그라운드 체크 기능 설정
upstage_ground_checker = UpstageGroundednessCheck()

# GraphState 클래스 정의
class GraphState(TypedDict):
    question: str       # 질문
    context: str        # 문서의 검색 결과
    answer: str         # 답변
    relevance: str      # 답변의 문서에 대한 관련성
    recursion_count: int  # 재귀 횟수

# 도구 함수 정의
def json_search_tool(input_text):
    docs = retriever.get_relevant_documents(input_text)
    if docs:
        summaries = [doc.page_content for doc in docs]
        return '\n\n'.join(summaries)
    else:
        return "해당하는 정보를 찾을 수 없습니다."

# Tool Definition
json_search_tool = Tool(
    name="JSONSearch",
    func=json_search_tool,
    description=(
        "이 도구를 사용하여 JSON 문서에서 특정 인물이나 이슈에 대한 정보를 검색하세요. "
        "회의번호, 회의명, 대수, 위원회명, 안건, 법률, 질의응답 유형, 문맥, 학습 문맥, "
        "문맥 요약, 질문자, 질문 등의 필드를 요약하여 제공합니다."
    )
)

# 도구 목록 생성
tools = [json_search_tool]

# LLM 정의
llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)

# 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 에이전트 초기화
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

# 시스템 프롬프트 설정
agent.agent.llm_chain.prompt.messages[0].prompt.template = (
    "한국어로 답변해 주세요. "
    "당신은 국회 JSON 문서에서 사용자에게 특정 인물이나 이슈에 대한 정보를 찾아주는 AI 비서입니다. "
    "사용자가 특정 인물이나 이슈에 대해 질문하면, JSONSearch 도구를 사용하여 해당 인물이 언급된 회의 정보를 찾아야 합니다. "
    "회의번호, 회의명, 대수, 위원회명, 안건, 법률, 질의응답 유형, 문맥, 학습 문맥, 문맥 요약, 질문자, 질문 등의 필드를 포함하여 요약된 응답을 제공하세요. "
    "JSON 문서에서 정보를 찾을 수 없다면, 정중하게 해당 정보가 없음을 알려주세요. "
    "도구의 사용 여부를 언급하지 말고 필요한 정보만 제공하세요."
)

/tmp/ipykernel_290/1721991243.py:39: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4", temperature=0, openai_api_key=openai_api_key)
/tmp/ipykernel_290/1721991243.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipykernel_290/1721991243.py:45: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.

In [5]:
# 그라운드 체크를 위한 함수 정의
def retrieve_document(state: GraphState) -> GraphState:
    # 재귀 횟수 증가 또는 초기화
    recursion_count = state.get('recursion_count', 0) + 1
    # 사용자의 질문에 따라 문서를 검색합니다.
    retrieved_docs = retriever.get_relevant_documents(state['question'])
    context = '\n\n'.join([doc.page_content for doc in retrieved_docs])
    # 기존의 'context'를 업데이트합니다.
    state.update({
        'context': context,
        'recursion_count': recursion_count
    })
    return state

def llm_answer(state: GraphState) -> GraphState:
    # 에이전트를 사용하여 답변을 생성합니다.
    answer = agent.run(input=state['question'])
    state.update({'answer': answer})
    return state

def relevance_check(state: GraphState) -> GraphState:
    # 그라운드 체크를 수행합니다.
    response = upstage_ground_checker.run(
        {"context": state['context'], "answer": state['answer']}
    )
    state.update({'relevance': response})
    return state

def is_relevant(state: GraphState) -> str:
    if state.get('relevance') == "grounded":
        return "관련성 O"
    elif state.get('relevance') == "notGrounded":
        return "관련성 X"
    else:
        return "확인불가"

# 그래프 워크플로우 정의
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)
workflow.add_node("relevance_check", relevance_check)

# 엣지 연결
workflow.add_edge("retrieve", "llm_answer")
workflow.add_edge("llm_answer", "relevance_check")

# 조건부 엣지 설정
workflow.add_conditional_edges(
    "relevance_check",
    is_relevant,
    {
        "관련성 O": END,
        "관련성 X": "retrieve",
        "확인불가": "retrieve",
    },
)

# 시작점 설정
workflow.set_entry_point("retrieve")

# 메모리 저장소 설정
memory_saver = MemorySaver()

# 워크플로우 컴파일
app = workflow.compile(checkpointer=memory_saver)

# 에이전트와 대화하는 함수 수정
def chat_with_agent(user_input):
    # 초기 상태 설정
    state = GraphState(
        question=user_input,
        context='',
        answer='',
        relevance='',
        recursion_count=0
    )
    config = RunnableConfig(recursion_limit=20, configurable={"thread_id": "SELF-RAG"})
    output = app.invoke(state, config=config)
    recursion_count = output.get('recursion_count', 0)
    relevance = output.get('relevance', '')
    # 최종 답변에 재귀 횟수와 그라운드 여부를 포함
    final_answer = output.get('answer', '')
    final_answer += f"\n\n총 재귀 횟수: {recursion_count}"
    if relevance == "grounded":
        final_answer += "\n그라운드 여부: O (관련 있음)"
    elif relevance == "notGrounded":
        final_answer += "\n그라운드 여부: X (관련 없음)"
    else:
        final_answer += "\n그라운드 여부: 확인 불가"
    return final_answer

In [6]:
# 예시 질문
user_input = "김영삼 대통령이 언급된 회의에 대한 정보를 알려주세요. 반드시 안건 이름과 날짜, 어떤 회의였는지, 몇 대 국회였는지 알려주세요. 질문자와 답변자에 대한 정보와 어떤 안건에 대해 이야기가 오고 갔는지 대화문도 알려주세요."
response = chat_with_agent(user_input)
print("Assistant:", response)

/tmp/ipykernel_290/1138666568.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(state['question'])
/tmp/ipykernel_290/1138666568.py:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = agent.run(input=state['question'])




> Entering new AgentExecutor chain...
```json
{
    "action": "JSONSearch",
    "action_input": "김영삼"
}
```
Observation: 판단되고 있다고 김영삼은 말했습니다.', 'context_summary': {'summary_q': '현재 국세 체납처분 위탁 조항이 추가된 이유 중 하나로는 산업기술 R&D 등에서 환수가 제대로 이루어지지 않는 상황이 지속되고 있다는 국감에서의 지적이 있었다고 이해하고 있는데, 실제로는 해당 조항을 통한 징수 사례가 없는 것으로 알고 있습니다. 이에 대한 이유는 무엇인가요?', 'summary_a': '국세 체납처분 위탁 조항은 산업기술 R&D 등에서의 환수 미흡에 대한 국감 지적을 고려하여 추가되었지만, 현재까지는 해당 조항을 통한 징수 사례가 발생하지 않아, 산자부장관의 적극적인 환수 노력이 부족한 상황으로 이해됩니다.'}, 'questioner_name': '김병관', 'questioner_ID': '686', 'questioner_ISNI': '0000000463651580', 'questioner_affiliation': '', 'questioner_position': '위원', 'question': {'tag': '180', 'comment': '그러니까 지금 국세청에 위탁하는 조항을 넣는 이유가 과학기술 쪽에서 R&D도 그렇고, 저희 산업기술 R&D에서도 환수가 제대로 안 이루어지고 있다는 지적들이 계속 있잖아요, 우리 국감 할 때마다.   그래서 사실은 산자부장관이 좀 더 적극적으로 환수 노력을 하게 되면 이런 부분이 필요가 없을 수 있는데, 계속적으로 국감에서 지적되고 있기 때문에 이런 내용이 들어가 있는 것 같거든요.', 'keyword': '환수,  국감,  국세청,  위탁,  조항,  이유,  과학기술,  쪽,  R&D'}, 'answerer_name': '김영삼', 'answerer_ID': ' ', 'answerer_

In [7]:
# 예시 질문
user_input = "그럼 다음 질문, 날짜상으로 마지막 회의에 대해 알려줘."
response = chat_with_agent(user_input)
print("Assistant:", response)



> Entering new AgentExecutor chain...
```json
{
    "action": "JSONSearch",
    "action_input": "김영삼 마지막 회의"
}
```
Observation: 'questioner_ISNI': '0000000463652524', 'questioner_affiliation': '', 'questioner_position': '위원', 'question': {'tag': '565', 'comment': '경기 군포시갑 출신 김정우 위원입니다.  오늘 국정감사 마지막 날인데요 부총리님을 비롯한 공직자분들 고생이 많으십니다. 또한 오늘이 20대 국회 마지막 국정감사의 마지막 날입니다. 그래서 아주 뜻깊은 날인데요. 그런데 성장률 속보치가 나와서 마음이 매우 무겁습니다.   그래서 먼저 이게 신상발언이기도 하고 의사진행발언이기도 한데 이 시간을 빌려서 좀 말씀드리겠습니다.   그동안 국정감사를 하면서 존경하는 야당 위원님들께서 경제가 어렵다는 말씀을 많이 하셨습니다. 경제 문제에 대해서도 많은 걱정을 해 주셨습니다. 그러한 야당 위원님들의 충정을 믿어 의심치 않습니다.   우리 기재위에는 서비스발전법 그리고 사회적경제 기본법 등 경제 활성화와 경제체질 개선을 위한 많은 법들이 지금 제안이 돼 있는데 아직 의결되지 못하고 잠들어 있습니다.  그래서 경제 활성화와 민생을 위한 일에는 여야가 따로 없다고 우리 수많은 선배 의원님들께서 말씀을 해 오셨습니다. 특히 오늘 많은 야당 위원님들께서 이제 협조하겠다고 말씀도 주셨습니다. 그래서 우리 기재위 야당 위원님들의 우리나라 경제를 걱정하는 마음을 법안 통과로 국민들에게 보여 줄 때가 됐다, 이제 국회가 나서야 된다 이렇게 생각합니다.  그래서 감히, 우리 위원회에는 나경원 원내대표도 계신데요 각 교섭단체 대표님과 지도부와 그리고 우리 기재위원들, 기재위 간사분들께 제안을 드립니다.  우리 기재위에 제안된 경제 활성화와 경제 체질 개선을 위한 법안을 신속하